In [21]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt 
import numpy as np
import baum_welch as bw
import cml_incomplete as cml_inc
import create_sequence_w_labels as cswl
from initial_guess import get_initial_values 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
def viterbi(V, a, b, initial_distribution):
    T = V.shape[0]
    M = a.shape[0]
 
    omega = np.zeros((T, M))
    omega[0, :] = np.log(initial_distribution * b[:, V[0]])
 
    prev = np.zeros((T - 1, M))
 
    for t in range(1, T):
        for j in range(M):
            # Same as Forward Probability
            probability = omega[t - 1] + np.log(a[:, j]) + np.log(b[j, V[t]])
 
            # This is our most probable state given previous state at time t (1)
            prev[t - 1, j] = np.argmax(probability)
 
            # This is the probability of the most probable state (2)
            omega[t, j] = np.max(probability)
 
    # Path Array
    S = np.zeros(T)
 
    # Find the most probable last hidden state
    last_state = np.argmax(omega[T - 1, :])
 
    S[0] = last_state
 
    backtrack_index = 1
    for i in range(T - 2, -1, -1):
        S[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1
 
    # Flip the path array since we were backtracking
    S = np.flip(S, axis=0)
 
    # Convert numeric values to actual hidden states
    result = []
    for s in S:
        result.append(s)
    return result


In [23]:
def combined_cml_approach(A, A_gt, B, B_gt, P, P_gt, pi, teoretical_observations, teoretical_labels, max_total_epoch, T_train, alternate_iter, K):
    prob_list_total = []
    for i in range(max_total_epoch):
        # Update A and P using the cml with incomplete label approach
        A, B, P = cml_inc.fit(A, B, P, pi, teoretical_observations[0:T_train], teoretical_labels[0:T_train], alternate_iter, update_A=True, update_B=False, update_P=True)
        
        # Update B and pi
        A, B, pi, prob_list, iter, final_tolerance = bw.fit(A, B, teoretical_observations[0:T_train], K, TOL = 1e-5, NUM_ITER = alternate_iter, pi = pi, hold_A=True)
        
        for j in range(len(prob_list)):
            prob_list_total.append(prob_list[j])
        print('Total epoch:', i)

    print('A_final:\n', np.round(A,3))
    print('A_gt:\n', A_gt)
    print('diff_A:', np.linalg.norm(A - A_gt))
    print('B_final:\n', np.round(B,3))
    print('B_gt:\n', B_gt)
    print('diff_B:', np.linalg.norm(B - B_gt))
    print('P_final:\n', np.round(P,3))
    print('P_gt:\n', np.round(P_gt,3))
    print('diff_P:', np.linalg.norm(P - P_gt))
    plt.plot(prob_list_total)
    plt.show()
    print('The best probability we can get on the combined CML appraoch is:', prob_list_total[-1])
    return A, B, pi

In [24]:
def pure_bw_approach(A, A_gt, B, B_gt, teoretical_observations, K, alternate_iter, pi):
    A, B, pi, prob_list, _, _ = bw.fit(A, B, teoretical_observations[0:T_train], K, TOL = 1e-5, NUM_ITER = alternate_iter, pi = pi, hold_A = False)
    print('A_final:\n', np.round(A,3))
    print('A_gt:\n', A_gt)
    print('diff_A:', np.linalg.norm(A - A_gt))
    print('B_final:\n', np.round(B,3))
    print('B_gt:\n', B_gt)
    print('diff_B:', np.linalg.norm(B - B_gt))
    plt.plot(prob_list)
    plt.show()
    print('The best probability we can get on the pure BW appraoch is:', prob_list[-1])
    return A, B, pi

In [33]:
def calculate_wrong_prediction(T_dev, T_train, teoretical_observations, teoretical_states, A_final_cml, B_final_cml, pi_final_cml, A_final_bw, B_final_bw, pi_final_bw):
    wrong_estimates_bw_total = 0
    wrong_estimates_cml_total = 0
    for dev_index in range(T_dev):
        result_cml = viterbi(np.array(teoretical_observations[T_train+dev_index]), A_final_cml, B_final_cml, pi_final_cml.T)
        result_bw = viterbi(np.array(teoretical_observations[T_train+dev_index]), A_final_bw, B_final_bw, pi_final_bw.T)
        wrong_estimates_cml = 0
        wrong_estimates_bw = 0
        for m in range(len(result_cml)):
            if result_cml[m] != teoretical_states[T_train+dev_index][m]:
                wrong_estimates_cml += 1
            if result_bw[m] != teoretical_states[T_train+dev_index][m]:
                wrong_estimates_bw += 1 
        wrong_estimates_bw_total += wrong_estimates_bw/N*100
        wrong_estimates_cml_total += wrong_estimates_cml/N*100
    
    wrong_estimates_cml_mean = np.round(wrong_estimates_cml_total/T_dev,4)
    wrong_estimates_bw_mean = np.round(wrong_estimates_bw_total/T_dev,4)
    cml_better = -np.round(wrong_estimates_cml_mean - wrong_estimates_bw_mean, 4)

    print('The percentage of the wrong estimated states for the combained cml appraoch in average is:', wrong_estimates_cml_mean)
    print('The percentage of the wrong estimated states for the bw appraoch in average is:', wrong_estimates_bw_mean)
    print('The cml in average is ' + str(cml_better) + ' better than the BW')

In [34]:
K = 2                  # states
L = 4                  # observations
T_train = 15           # number of training sequences 
T_dev = 10             # number of development set sequences
T = T_train + T_dev    # total number of sequences
N = 2048               # number of observations on each sequence
noise_variance = 0.1   # Noise variance
alternate_iter = 3     # alternate the update of A and P after 'alternate_iter' iterations with B and pi
max_total_epoch = 5    # Do the update of A and P and B and pi respectively max_total_epoch times 
                       # where each iteration of A/B/P/pi is alternate_iter times
bw_iter = 40           # Max iterate for tthe pure baum-welch approach 
do_experiment = 5      

for i in range(do_experiment):
    A_gt, B_gt, pi_gt, P_gt, teoretical_observations, teoretical_states, teoretical_labels = cswl.hmmgenerate(K, L, T, N, S=0, D=0, edhmm = False)
    A_init, B_init, pi_init, P_init = get_initial_values(A_gt, B_gt, noise_variance, P_gt, have_P = True)
    A_final_cml, B_final_cml, pi_final_cml = combined_cml_approach(A_init, A_gt, B_init, B_gt, P_init, P_gt, pi_init, teoretical_observations, teoretical_labels, max_total_epoch, T_train, alternate_iter, K)
    A_final_bw, B_final_bw, pi_final_bw = pure_bw_approach(A_init, A_gt, B_init, B_gt, teoretical_observations, K, bw_iter, pi_init)
    calculate_wrong_prediction(T_dev, T_train, teoretical_observations, teoretical_states, A_final_cml, B_final_cml, pi_final_cml, A_final_bw, B_final_bw, pi_final_bw)


SystemError: CPUDispatcher(<function incomplete_label_forward at 0x0000019F99E22D30>) returned a result with an error set